In [1]:

from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import HashingTF, IDF, Tokenizer


conf = SparkConf()
conf.setMaster("local[*]").setAppName("CENG790-Project")

spark = SparkSession.builder.config(conf=SparkConf()).getOrCreate()
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")

22/01/15 22:41:19 WARN Utils: Your hostname, akad01 resolves to a loopback address: 127.0.1.1; using 10.1.46.203 instead (on interface wlp48s0)
22/01/15 22:41:19 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/01/15 22:41:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:

# # Saving the oscar dataset(28GB) to json format. (only execute once)
# from datasets import load_from_disk
# dataset = load_from_disk("lang_detected")["train"]
# # Set num_proc according to your cpu count, num_proc=20 means 20 thread will be executed paralelly.
# dataset.to_json("dataset_json", num_proc=20)

In [5]:
# Reads from the dataset_json.json file, 
df_json = spark.read.json("dataset.json")

In [6]:
small_df = df_json.limit(100)
small_df.show(10)
small_df.write.json("dataset_small.json")

+---+----+--------------------+
| id|lang|                text|
+---+----+--------------------+
|  0|  tr|Son yıllarda görü...|
|  1|  tr|Şehrin karmaşası ...|
|  2|  tr|2010 Yılında Mard...|
|  3|  tr|29Ekim 2009 2010 ...|
|  4|  tr|Yüksek İslam Şura...|
|  5|  tr|Oncelıkle bu etkı...|
|  6|  tr|Mavi-Mi Sanat Mer...|
|  7|  tr|Türkiye Futbol Fe...|
|  8|  tr|anlami-nedir.com'...|
|  9|  tr|Kepez Belediye Ba...|
+---+----+--------------------+
only showing top 10 rows



In [ ]:

tokenizer = Tokenizer(inputCol="text", outputCol="words")
wordsData = tokenizer.transform(rdd_json)

# TODO: Change numFeatures=20.
hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures")
featurizedData = hashingTF.transform(wordsData)
# alternatively, CountVectorizer can also be used to get term frequency vectors

idf = IDF(inputCol="rawFeatures", outputCol="features")
idfModel = idf.fit(featurizedData)
rescaledDataWithLang = idfModel.transform(featurizedData)


In [ ]:
from pyspark.ml.feature import StringIndexer

indexer = StringIndexer(inputCol="lang", outputCol="label")
rescaledData = indexer.fit(rescaledDataWithLang).transform(rescaledDataWithLang)


In [ ]:
rescaledData.count()


In [ ]:
rescaledData.select("label", "features").show(1, False)

In [ ]:
(trainingData, testData) = rescaledData.randomSplit([0.8, 0.2])

In [ ]:

from pyspark.ml.regression import LinearRegression

lr = LinearRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8, featuresCol="features", labelCol="label")

# Fit the model
lrModel = lr.fit(trainingData)

# Print the coefficients and intercept for linear regression
print("Coefficients: %s" % str(lrModel.coefficients))
print("Intercept: %s" % str(lrModel.intercept))

# Summarize the model over the training set and print out some metrics
trainingSummary = lrModel.summary
print("numIterations: %d" % trainingSummary.totalIterations)
print("objectiveHistory: %s" % str(trainingSummary.objectiveHistory))
trainingSummary.residuals.show()
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)

In [ ]:





##


# Load training data


lr = LinearRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8, featuresCol="text", predictionCol="lang")

# Fit the model
lrModel = lr.fit(training)

# Print the coefficients and intercept for linear regression
print("Coefficients: %s" % str(lrModel.coefficients))
print("Intercept: %s" % str(lrModel.intercept))

# Summarize the model over the training set and print out some metrics
trainingSummary = lrModel.summary
print("numIterations: %d" % trainingSummary.totalIterations)
print("objectiveHistory: %s" % str(trainingSummary.objectiveHistory))
trainingSummary.residuals.show()
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)